<a href="https://colab.research.google.com/github/andrewhywong/USDAsymbols/blob/main/USDAsymbols.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from urllib import parse
import bs4
import re
from multiprocessing import Process
from multiprocessing import Manager
import requests
import time
import xlrd
from bs4 import BeautifulSoup
from lxml import etree
import re
import pandas as pd
import json

In [ ]:
# this database url should be somewhere in the inspection of normal JS page
# Not working for apparent page 
root_url = 'https://plantsservices.sc.egov.usda.gov/api/PlantProfile?symbol='
# symbolIDList = ['LUFO2','LAAN81','CIIN']
symbolIDList = pd.read_csv('/content/drive/MyDrive/NYMPHS/All_uni_symbols.csv')

In [ ]:
class getMain(object):
  def __init__(self, i):
    print(i)
    # prepare headers
    self.headers = {
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
      'Accept-Encoding': 'gzip, deflate, br',
      'Accept-Language': 'en,zh-CN;q=0.9,zh;q=0.8,ko;q=0.7',
      'Cache-Control': 'max-age=0',
      'Connection': 'keep-alive',
      'Cookie': '_ga=GA1.2.1272093178.1627507535; __utma=77627408.1272093178.1627507535.1627507535.1627507535.1; __utmz=77627408.1627507535.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); nmstat=7d00aeea-6429-c4f4-db3f-d22bbdd2ff45; _4c_=XVLNbts8EHwVg4eerB%2Fqn74VDlD00BzSFh%2B%2Bk0CRa4mwIgokbSUN%2FO7ZtZXAiC9ezs7OkqN5Y8sAE9vxKhNZxsui5nW6ZUd49Wz3xpzR9HdmOwa5KOqmKyNk6qg4cBmJXHTRoVCqFg3vQDVsy15IKy%2BzJi%2FzMhX5Zcs06t80HGjwpp%2FuebixKNMSeeaDJr%2F0S84L7LtlJXw2qkbgZe%2BphCBVzSv1jSmrATW5iKu4QuHwD09RnqZYw0TrZqexPrkR6yGE2e%2BSZFmWeJLexyevZdzbc%2FIEHqRTQysn3f5WBiYFyd7ZeUQAuyNID4mZNLzE8zCj4o%2Fv7d%2BfD7Q7qzO0gtdNjD7XZVqjN0iYndUnFdrwOtMVF%2Bg2Xh%2BxoeFsFLSL0WGg%2BTKnm6%2FoAKYfAsJNVRA6O6LEGSkuuN8uXwdX9HOwLmnQnzpvAuxXf3japALhR1g2T98eNtRD%2FBGhztnFA63ZD84%2Bw0ZkiFpMCPvvKu3x6OAAzl1Z9yb21vYjxMo%2BJ7TzJvph6opg2hA8%2BOgAOjqR2EzfhedYjVbJkWYwpVv2x5m%2BB%2FcLwmAxmXiW2gRjJzmSFRi0%2B4xpeiyejsHOd7AHRSOt7BVMHvfdXvkB02dH6H%2BGObpGKxN5WudN1dQC4xMwJuh8Sr%2FL5fIO',
      'Host': 'plantsservices.sc.egov.usda.gov',
      'sec-ch-ua': '"Google Chrome";v="95", "Chromium";v="95", ";Not A Brand";v="99"',
      'sec-ch-ua-mobile': '?0',
      'sec-ch-ua-platform': '"Windows"',
      'Sec-Fetch-Dest': 'document',
      'Sec-Fetch-Mode': 'navigate',
      'Sec-Fetch-Site': 'none',
      'Sec-Fetch-User': '?1',
      'Upgrade-Insecure-Requests': '1',
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
        }
  def craw(self,full_url,i):
    s = requests.get(full_url,headers=self.headers)
    soup = BeautifulSoup(s.text, 'html.parser')
    getList = soup.find_all("scientificname")
    if getList:
      # find anything between pattern:
      # <scientificname>&lt;i&gt; + &lt;/i&gt;
      # of the last element of this soup list
      targetLineStr = str(getList[-1])
      start = targetLineStr.find("<scientificname>&lt;i&gt;") + len("<scientificname>&lt;i&gt;")
      end = targetLineStr.find("&lt;/i&gt;")
      sciName = targetLineStr[start:end]
    else:
      sciName = "NA"
    return sciName

In [ ]:
speciesdf = pd.DataFrame()
for i in symbolIDList.iloc[:,0].values:
  full_url = root_url + i
  obj = getMain(i)
  sciname = obj.craw(full_url,i)
  lst = []
  for r in range(1):
    lst.append([i,sciname])
  speciesdf = speciesdf.append(lst)

speciesdf = speciesdf.rename(columns={0:'Symbol',1:'SciName'})

In [ ]:
speciesdf.to_csv('/content/drive/MyDrive/NYMPHS/All_uni_symbols_SciName.csv', index=False)